In [ ]:
# 血淋淋的教训是记住权值更新w+=！！！
#Reference: https://iamtrask.github.io/2015/11/15/anyone-can-code-lstm/
#https://blog.csdn.net/weiwei9363/article/details/78902455（解读）

In [3]:
import copy, numpy as np
np.random.seed(0)

# compute sigmoid nonlinearity
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

# convert output of sigmoid function to its derivative
def sigmoid_output_to_derivative(output):
    return output*(1-output)


# training dataset generation
int2binary = {}
binary_dim = 8

largest_number = pow(2,binary_dim)
binary = np.unpackbits(
    np.array([range(largest_number)],dtype=np.uint8).T,axis=1)
for i in range(largest_number):
    int2binary[i] = binary[i]

# input variables
alpha = 0.1
n_input = 2
n_hidden= 16
n_output = 1


# initialize neural network weights
#使用2为输入维度
#random.random生成数是【0，1），进而保证w在【-1，1）
w1 = 2*np.random.random((n_input,n_hidden)) - 1
w2 = 2*np.random.random((n_hidden,n_output)) - 1
wh = 2*np.random.random((n_hidden,n_hidden)) - 1

w_update1 = np.zeros_like(w1)
w_update2 = np.zeros_like(w2)
w_updateh = np.zeros_like(wh)

In [4]:
for j in range(10000):#训练次数
    
    #generate a simple addtion problem(a+b=c???)
    #除以2为了相加的值小于等于256
    a_int=np.random.randint(largest_number/2)#int version
    a=int2binary[a_int]#binary coding

    b_int=np.random.randint(largest_number/2)#int version
    b=int2binary[b_int]#binary coding
    
    c_int=a_int+b_int
    c=int2binary[c_int]
    
    #where to store the best guess
    d=np.zeros_like(c)
    
    overallError=0#每次训练之后把总误差清零，用来记录下一次训练的情况

    layer_2_deltas=list()
    layer_1_values=list()
    layer_1_values.append(np.zeros(n_hidden))
#     print(j,": ",wh[1])
    
    #moving along the positions int the binary encoding
    for position in range(binary_dim):
       
        X=np.array([[a[binary_dim-position-1],b[binary_dim-position-1]]])
        
        Y=np.array([[c[binary_dim-position-1]]]).T

        #print(layer_1_values[-1])#负数表示倒数第几个数，
        #至于为什么是0.，0和0. 0还需要继续思考：我现在的想法是为了取上一次的layer1的值
        #hidden_layer( input +prev_hidden)
        layer_1=sigmoid(np.dot(X,w1)+np.dot(layer_1_values[-1],wh))
        #加的是之前的s3
        
        #output layer(new binary representation
        layer_2=sigmoid(np.dot(layer_1,w2))
 
        layer_2_error=Y-layer_2     
   
        layer_2_deltas.append((layer_2_error)*sigmoid_output_to_derivative(layer_2))
        #print (layer_2_deltas)
        overallError+=np.abs(layer_2_error[0])
        
        #decode estimate so we can print it out,
        #         返回四舍五入之后的值，所以之后要么是1要么是0
        #d装的是某个位置的值，也就是训练之后的值

        d[binary_dim - position - 1]=np.round(layer_2[0][0])
    
        # store hidden layer so we can use it in the next timestep
        layer_1_values.append(copy.deepcopy(layer_1))
    future_layer_1_delta=np.zeros(n_hidden)

    for position in range(binary_dim):
        X=np.array([[a[position],b[position]]])
        layer_1=layer_1_values[-position-1]
        prev_layer_1=layer_1_values[-position-2]
        
        #error at output layer
        layer_2_delta=layer_2_deltas[-position-1]
        #error at hidden layer
        layer_1_delta=(future_layer_1_delta.dot(wh.T)+\
            layer_2_delta.dot(w2.T))*sigmoid_output_to_derivative(layer_1)
                                
            
        #update the weights,atleat_xd()直接将数组视为x维的数
        w_update2+=np.atleast_2d(layer_1).T.dot(layer_2_delta)
        w_updateh+=np.atleast_2d(prev_layer_1).T.dot(layer_1_delta)
        w_update1+=X.T.dot(layer_1_delta)
        
        future_layer_1_delta=layer_1_delta

    w1+=w_update1*alpha
    w2+=w_update2*alpha
    wh+=w_updateh*alpha

    #恢复1原来的δ，也就是清零
    w_update1*=0
    w_update2*=0
    w_updateh*=0
    
    #print out progress
    if(j%1000==0):
        print("Error:"+str(overallError))
        print("Pred:"+str(d))
        print("True:"+str(c))
        out =0
        for index,x in enumerate(reversed(d)):
            #使用reversed为了使训练方便，因为训练的输入顺序是从低位开始的，现实倒过来
            out+=x*pow(2,index)
        print(str(a_int)+"+"+str(b_int)+"="+str(out))
        print("---------------------------------------------")

Error:[3.45638663]
Pred:[0 0 0 0 0 0 0 1]
True:[0 1 0 0 0 1 0 1]
9+60=1
---------------------------------------------
Error:[3.63389116]
Pred:[1 1 1 1 1 1 1 1]
True:[0 0 1 1 1 1 1 1]
28+35=255
---------------------------------------------
Error:[3.91366595]
Pred:[0 1 0 0 1 0 0 0]
True:[1 0 1 0 0 0 0 0]
116+44=72
---------------------------------------------
Error:[3.72191702]
Pred:[1 1 0 1 1 1 1 1]
True:[0 1 0 0 1 1 0 1]
4+73=223
---------------------------------------------
Error:[3.5852713]
Pred:[0 0 0 0 1 0 0 0]
True:[0 1 0 1 0 0 1 0]
71+11=8
---------------------------------------------
Error:[2.53352328]
Pred:[1 0 1 0 0 0 1 0]
True:[1 1 0 0 0 0 1 0]
81+113=162
---------------------------------------------
Error:[0.57691441]
Pred:[0 1 0 1 0 0 0 1]
True:[0 1 0 1 0 0 0 1]
81+0=81
---------------------------------------------
Error:[1.42589952]
Pred:[1 0 0 0 0 0 0 1]
True:[1 0 0 0 0 0 0 1]
4+125=129
---------------------------------------------
Error:[0.47477457]
Pred:[0 0 1 1 1 0 0 0